In [ ]:
#use kiwi or konlppy

In [1]:
#kiwi
!pip install kiwipiepy

In [8]:
from kiwipiepy import Kiwi

kiwi = Kiwi()
text = "김민수는 서울대학교에 재학 중입니다."

tokens = kiwi.tokenize(text)

for t in tokens:
    print(t.form, t.tag)


김민수 NNP
는 JX
서울대 NNP
학교 NNG
에 JKB
재학 NNG
중 NNB
이 VCP
ᆸ니다 EF
. SF


In [ ]:
#making pipeline

def kiwi_preprocess(text):
    tokens = kiwi.tokenize(text)
    return " ".join([t.form for t in tokens])


In [10]:
import spacy

nlp = spacy.load("ko_core_news_lg")

clean_text = kiwi_preprocess(text)
doc = nlp(clean_text)

for ent in doc.ents:
    print(ent.text, ent.label_)


김민수 PS
서울대 OG


In [6]:
doc

김민수 는 서울대 학교 에 재학 중 이 ᆸ니다 .

In [ ]:
from kiwipiepy import Kiwi

kiwi = Kiwi()
sample_text = """
어제 김철수 부장은 박민수 대리와 함께 삼성전자 본사인 서울 강남구에서 회사 서버 사이트 운영 
현황에 대한 회의를 진행하였다. 현재 내부 시스템은 intranet.company.co.kr 도메인을 사용하고 있으며, 
접속 기록에는 IP 주소 10.0.3.45가 확인되었다. 본 프로젝트의 총 예산은 5억 원이며, 
완료 목표일은 2026년 3월 15일 오전 10시로 설정되어 있다. 내부 테스트 계정의 비밀번호는 
TestAccount2026으로 관리 중이다.

추가로, 이순신 과장은 LG유플러스 및 SK텔레콤 관계자들과 협업하여 데이터 백업 정책을 검토 중이다. 
지난 2026년 2월 3일 오후 2시부터 4시까지, 부산 데이터 센터에서 장애가 발생하여 일부 내부 재무 자료 
접근이 제한되었다. 현재 확인된 관련 계좌번호는 321-54-987654이며, 보안팀에서 해당 건에 대한 조사를 
진행하고 있다."""

tokens = kiwi.tokenize(sample_text)

for t in tokens:
    print(t.form, t.tag)

In [7]:
def kiwi_preprocess(sample_text):
    tokens = kiwi.tokenize(sample_text)
    return " ".join([t.form for t in tokens])

In [ ]:
import spacy

nlp = spacy.load("ko_core_news_lg")

clean_text = kiwi_preprocess(sample_text)
doc = nlp(clean_text)

for ent in doc.ents:
    print(ent.text, ent.label_)

In [22]:
#declare sensitive keyword
SENSITIVE_KEYWORDS = [
    "비밀번호", "패스워드", "도메인", "IP 주소", "키"
]

def is_sensitive_context(text):
    return any(k in text for k in SENSITIVE_KEYWORDS)


In [23]:
PASSWORD_REGEX = r"\b[A-Za-z0-9@#$%^&+=!]{8,}\b"
IP_REGEX = r"\b(?:\d{1,3}\.){3}\d{1,3}\b"
DOMAIN_REGEX = r"\b[a-zA-Z0-9.-]+\.(com|net|org|kr|io)\b"

In [24]:
def kiwi_preprocess(text):
    tokens = kiwi.tokenize(text)
    return " ".join([t.form for t in tokens])


In [25]:
def has_sensitive_ner(text):
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ in ["PS", "OG", "LC", "DT", "QT"]:
            return True
    return False


In [26]:
def mask_matches(text, matches):
    masked_text = text
    for m in set(matches):
        masked_text = masked_text.replace(m, "*" * len(m))
    return masked_text


In [27]:
#full pipeline

import re 

def dlp_scan(text):
    # Step 1: KIWI preprocess
    clean_text = kiwi_preprocess(text)

    # Step 2: Context detection
    context_flag = is_sensitive_context(clean_text)
    ner_flag = has_sensitive_ner(clean_text)

    if not context_flag and not ner_flag:
        return text, False

    # Step 3: Regex extraction
    password_matches = re.findall(PASSWORD_REGEX, clean_text)
    ip_matches = re.findall(IP_REGEX, clean_text)
    domain_matches = re.findall(DOMAIN_REGEX, clean_text)

    all_matches = password_matches + ip_matches + domain_matches

    if not all_matches:
        return text, False

    # Step 4: Mask
    masked_text = mask_matches(clean_text, all_matches)

    return masked_text, True



In [28]:
sample_text = """
어제 김철수 부장은 박민수 대리와 함께 삼성전자 본사인 서울 강남구에서 회사 서버 사이트 운영 
현황에 대한 회의를 진행하였다. 현재 내부 시스템은 intranet.company.co.kr 도메인을 사용하고 있으며, 
접속 기록에는 IP 주소 10.0.3.45가 확인되었다. 본 프로젝트의 총 예산은 5억 원이며, 
완료 목표일은 2026년 3월 15일 오전 10시로 설정되어 있다. 내부 테스트 계정의 비밀번호는 
TestAccount2026으로 관리 중이다.

추가로, 이순신 과장은 LG유플러스 및 SK텔레콤 관계자들과 협업하여 데이터 백업 정책을 검토 중이다. 
지난 2026년 2월 3일 오후 2시부터 4시까지, 부산 데이터 센터에서 장애가 발생하여 일부 내부 재무 자료 
접근이 제한되었다. 현재 확인된 관련 계좌번호는 321-54-987654이며, 보안팀에서 해당 건에 대한 조사를 
진행하고 있다."""

result, flagged = dlp_scan(sample_text)
print(flagged)
print(result)


True
어제 김철수 부장 은 박민수 대리 와 함께 삼성전자 본사 이 ᆫ 서울 강남구 에서 회사 서버 사이트 운영 현황 에 대하 ᆫ 회의 를 진행 하 었 다 . 현재 내부 시스템 은 ******** . co mpany.co.** 도메인 을 사용 하 고 있 으며 , 접속 기록 에 는 IP 주소 ********* 가 확인 되 었 다 . 본 프로젝트 의 총 예산 은 5 억 원 이 며 , 완료 목표 일 은 2026 년 3 월 15 일 오전 10 시 로 설정 되 어 있 다 . 내부 테스트 계정 의 비밀번호 는 *********** 2026 으로 관리 중 이 다 . 추가 로 , 이순신 과장 은 LG유플러스 및 SK 텔레콤 관계자 들 과 협업 하 어 데이터 백업 정책 을 검토 중 이 다 . 지나 ᆫ 2026 년 2 월 3 일 오후 2 시 부터 4 시 까지 , 부산 데이터 센터 에서 장애 가 발생 하 어 일부 내부 재무 자료 접근 이 제한 되 었 다 . 현재 확인 되 ᆫ 관련 계좌 번호 는 321-54-987654 이 며 , 보안 팀 에서 해당 건 에 대하 ᆫ 조사 를 진행 하 고 있 다 .


In [31]:
#declare sensitive keyword
SENSITIVE_KEYWORDS = [
    "비밀번호", "패스워드", "도메인", "IP 주소", "키"
]

def is_sensitive_context(text):
    return any(k in text for k in SENSITIVE_KEYWORDS)


In [32]:
def extract_regex_entities(text):
    results = []
    results += re.findall(PASSWORD_REGEX, text)
    results += re.findall(IP_REGEX, text)
    results += re.findall(DOMAIN_REGEX, text)
    return results


In [40]:
#declare sensitive ner
SENSITIVE_NER_LABELS = [
    "PC",
    "OG",
    "LC"
]

In [44]:
def extract_ner_entities(text):
    doc = nlp(text)
    return [
        ent.text
        for ent in doc.ents
        if ent.label_ in SENSITIVE_NER_LABELS
    ]


In [45]:
def mask_text(text, targets):
    masked = text
    for t in set(targets):
        masked = masked.replace(t, "*" * len(t))
    return masked


In [49]:
def kiwi_preprocess(text):
    tokens = kiwi.tokenize(text)
    return " ".join([t.form for t in tokens])

In [50]:
def dlp_scan(text):
    clean_text = kiwi_preprocess(text)

    # Step 1: context check
    if not is_sensitive_context(clean_text):
        return text, False

    # Step 2: extract targets
    regex_targets = extract_regex_entities(clean_text)
    ner_targets = extract_ner_entities(clean_text)

    all_targets = regex_targets + ner_targets

    if not all_targets:
        return text, False

    # Step 3: mask
    masked_text = mask_text(clean_text, all_targets)

    return masked_text, True


In [51]:
text = "김민수의 계정 비밀번호는 TestAccount2026이다."

r = dlp_scan(text)
print(r)

('김민수 의 계정 비밀번호 는 *********** 2026 이 다 .', True)
